# Autoencoder Implemented in Tensorflow

In [ ]:
%matplotlib inline
from __future__ import division, print_function, absolute_import
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import os

from sklearn.cross_validation import train_test_split

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=False)

# Parameters
learning_rate = 0.01
training_epochs = 200
batch_size = 256
display_step = 10
examples_to_show = 10

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 256 # 2nd layer num features
n_hidden_3 = 2048
encoded_dim = 128
n_input = 784 # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([n_hidden_2, encoded_dim])),
    'decoder_h1': tf.Variable(tf.random_normal([encoded_dim, n_hidden_2])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h3': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([encoded_dim])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b2': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b3': tf.Variable(tf.random_normal([n_input])),
}

# Encoder
def encoder(x):
    # layer_1 encoding
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights["encoder_h1"]), 
                               biases["encoder_b1"]))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights["encoder_h2"]),
                                biases["encoder_b2"]))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights["encoder_h3"]),
                                biases["encoder_b3"]))
#     layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights["encoder_h4"]),
#                                 biases["encoder_b4"]))
    return layer_3

# Decoder
def decoder(x):
    # layer_1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights["decoder_h1"]),
                               biases["decoder_b1"]))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights["decoder_h2"]),
                               biases["decoder_b2"]))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights["decoder_h3"]),
                               biases["decoder_b3"]))
#     layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights["decoder_h4"]),
#                                biases["decoder_b4"]))
    return layer_3

for d in ["/gpu:0", "/gpu:1", '/gpu:2', '/gpu:3']:
    with tf.device(d):
        encoder_op = encoder(X)
        decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Groundtruths
y_true = X

# Loss functions
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

sess =  tf.Session()
sess.run(init)
total_batch = int(mnist.train.num_examples/batch_size)
# Training
for epoch in range(training_epochs):
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Run optimization
        _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
    # Display logs per epoch step
    if epoch % display_step == 0:
        print("Epoch:", "%04d" % (epoch+1), "cost:", "{:.9f}".format(c))

print("Optimization Finished!")

In [ ]:
# Applying encode and decode over test set
encode_decode = sess.run(
            y_pred, feed_dict={X: mnist.test.images[:examples_to_show]})
# Compare original images with their reconstructions
f, a = plt.subplots(2, 10, figsize=(10, 2))
for i in range(examples_to_show):
    a[0][i].get_xaxis().set_visible(False)
    a[0][i].get_yaxis().set_visible(False)
    a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
    a[1][i].get_xaxis().set_visible(False)
    a[1][i].get_yaxis().set_visible(False)
    a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
f.show()

In [ ]:
encoded = sess.run(encoder_op, feed_dict={X: mnist.test.images})

In [ ]:
encoded.shape

In [ ]:
# Get the error of clustering

from scipy.spatial.distance import cdist

class TMM(object):
    def __init__(self, n_components=1, alpha=1):
        self.n_components = n_components
        self.tol = 1e-5
        self.alpha = float(alpha)

    def fit(self, X):
        from sklearn.cluster import KMeans
        kmeans = KMeans(self.n_components, n_init=20)
        kmeans.fit(X)
        self.cluster_centers_ = kmeans.cluster_centers_
        self.covars_ = np.ones(self.cluster_centers_.shape)

    def transform(self, X):
        p = 1.0
        dist = cdist(X, self.cluster_centers_)
        r = 1.0/(1.0+dist**2/self.alpha)**((self.alpha+p)/2.0)
        r = (r.T/r.sum(axis=1)).T
        return r

    def predict(self, X):
        return self.transform(X).argmax(axis=1)

tmm = TMM(n_components=10)

def calc_dist_error(X):
    tmm = TMM(n_components=10)
    l = []
    tmm.fit(X)
    q = tmm.transform(X)
    q = (q.T/q.sum(axis=1)).T
    p = (q**2)
    p = (p.T/p.sum(axis=1)).T
    grad = 2.0/(1.0+cdist(X, tmm.cluster_centers_, 
        'sqeuclidean'))*(p-q)*cdist(X, tmm.cluster_centers_, 'cityblock')
    return p, q, grad    

In [ ]:
p, q, grad = calc_dist_error(encoded)

In [ ]:
print(grad.shape)
print(grad[:5])